# Importing packages

In [1]:
# Install the tensorflow module responsible for the optimization (if not installed)
!pip install tensorflow_model_optimization

     |████████████████████████████████| 211 kB 5.1 MB/s 


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import random

# Preparing the data

In [3]:
# Generate batches of random values
def randomSignal(number_batches, timesteps_per_batch=32):
  signal = []

  for i in range(number_batches):
    l=[]
    for j in range(timesteps_per_batch):
      l.append((random.random()-0.5)*2)
    signal.append(l)

  return signal

In [4]:
nbatches = 1000000    # Number of batches
timesteps = 32       # Number of values in each batch
data = randomSignal(nbatches, timesteps_per_batch=timesteps)
data = np.array(data)

In [5]:
# Calculate the Fast Fourier Transform of the generated data
# This is the function we will try to imitate using NN (just for fun !)
ffts = []
for i in data:
  x=np.fft.rfft(i)
  ffts.append(x)

In [6]:
ffts_real=[]   # The real parts of the ffts
ffts_imag=[]   # The imaginary parts of the ffts
for i in ffts:
  y=[]
  x=[]
  for j in i:
    y.append(j.real)
    x.append(j.imag)
  ffts_real.append(y)
  ffts_imag.append(x)

ffts_real = np.array(ffts_real)
ffts_imag = np.array(ffts_imag)

In [7]:
# Calculate the ground truth output of the FFT function (labels)
ffts_test = np.fft.rfft(data)

In [8]:
# decompose the test ffts to real parts and imaginary parts
ffts_real_test = ffts_test.real
ffts_imag_test = ffts_test.imag

# Training traditional models

In [9]:
# Model for learning the real part of the ffts
model_real = Sequential()
model_real.add(Dense(int(timesteps/2)+1, use_bias=False))

model_real.compile(optimizer='adam',loss='mse')
model_real.fit(data, ffts_real, epochs=10, batch_size=1024, validation_split=0.3)

Epoch 1/10
684/684 [==============================] - 2s 2ms/step - loss: 3.1107 - val_loss: 1.1280
Epoch 2/10
684/684 [==============================] - 1s 2ms/step - loss: 0.4777 - val_loss: 0.1255
Epoch 3/10
684/684 [==============================] - 1s 2ms/step - loss: 0.0435 - val_loss: 0.0069
Epoch 4/10
684/684 [==============================] - 1s 2ms/step - loss: 0.0019 - val_loss: 1.3384e-04
Epoch 5/10
684/684 [==============================] - 1s 2ms/step - loss: 2.7310e-05 - val_loss: 5.8619e-07
Epoch 6/10
684/684 [==============================] - 1s 2ms/step - loss: 8.7981e-08 - val_loss: 3.0928e-10
Epoch 7/10
684/684 [==============================] - 1s 2ms/step - loss: 4.9992e-11 - val_loss: 1.6855e-11
Epoch 8/10
684/684 [==============================] - 1s 2ms/step - loss: 1.2711e-11 - val_loss: 9.4639e-12
Epoch 9/10
684/684 [==============================] - 1s 2ms/step - loss: 7.0624e-12 - val_loss: 5.0950e-12
Epoch 10/10
684/684 [==============================] - 1

In [10]:
# Model for learning the imaginary part of the ffts
model_imag = Sequential()
model_imag.add(Dense(int(timesteps/2)+1, use_bias = False))

model_imag.compile(optimizer='adam',loss='mse')
model_imag.fit(data, ffts_imag, epochs=10, batch_size=1024, validation_split=0.3)

Epoch 1/10
684/684 [==============================] - 2s 2ms/step - loss: 2.3993 - val_loss: 0.8342
Epoch 2/10
684/684 [==============================] - 1s 2ms/step - loss: 0.3449 - val_loss: 0.0859
Epoch 3/10
684/684 [==============================] - 1s 2ms/step - loss: 0.0291 - val_loss: 0.0043
Epoch 4/10
684/684 [==============================] - 1s 2ms/step - loss: 0.0011 - val_loss: 7.5784e-05
Epoch 5/10
684/684 [==============================] - 1s 2ms/step - loss: 1.5136e-05 - val_loss: 2.9131e-07
Epoch 6/10
684/684 [==============================] - 1s 2ms/step - loss: 4.2733e-08 - val_loss: 1.3551e-10
Epoch 7/10
684/684 [==============================] - 1s 2ms/step - loss: 2.8102e-11 - val_loss: 1.3046e-11
Epoch 8/10
684/684 [==============================] - 1s 2ms/step - loss: 9.8030e-12 - val_loss: 7.1537e-12
Epoch 9/10
684/684 [==============================] - 1s 2ms/step - loss: 5.3795e-12 - val_loss: 3.8608e-12
Epoch 10/10
684/684 [==============================] - 1

# Training Quantization Aware Training Models

In [11]:
import tensorflow_model_optimization as tfmot

quantize_model_real = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model_real = quantize_model_real(model_real)

# The QAT model requires a recompile.
q_aware_model_real.compile(optimizer='adam',
              loss="mse")

q_aware_model_real.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLaye (None, 32)                3         
_________________________________________________________________
quant_dense (QuantizeWrapper (None, 17)                549       
Total params: 552
Trainable params: 544
Non-trainable params: 8
_________________________________________________________________


In [12]:
quantize_model_imag = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model_imag = quantize_model_imag(model_imag)

# The QAT model requires a recompile.
q_aware_model_imag.compile(optimizer='adam',
              loss="mse")

q_aware_model_imag.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_1 (QuantizeLa (None, 32)                3         
_________________________________________________________________
quant_dense_1 (QuantizeWrapp (None, 17)                549       
Total params: 552
Trainable params: 544
Non-trainable params: 8
_________________________________________________________________


In [13]:
# Train the QAT model (real part)
q_aware_model_real.fit(data, ffts_real, batch_size=1024, epochs=10, validation_split=0.3)

Epoch 1/10
684/684 [==============================] - 2s 3ms/step - loss: 0.0092 - val_loss: 0.0025
Epoch 2/10
684/684 [==============================] - 2s 3ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 3/10
684/684 [==============================] - 2s 3ms/step - loss: 0.0010 - val_loss: 9.2732e-04
Epoch 4/10
684/684 [==============================] - 2s 3ms/step - loss: 9.2707e-04 - val_loss: 9.2192e-04
Epoch 5/10
684/684 [==============================] - 2s 3ms/step - loss: 9.1220e-04 - val_loss: 9.1183e-04
Epoch 6/10
684/684 [==============================] - 2s 3ms/step - loss: 9.1012e-04 - val_loss: 8.9679e-04
Epoch 7/10
684/684 [==============================] - 2s 3ms/step - loss: 9.1002e-04 - val_loss: 9.0937e-04
Epoch 8/10
684/684 [==============================] - 2s 3ms/step - loss: 9.1149e-04 - val_loss: 9.0701e-04
Epoch 9/10
684/684 [==============================] - 2s 3ms/step - loss: 9.0990e-04 - val_loss: 8.9366e-04
Epoch 10/10
684/684 [===========================

In [14]:
# Train the QAT model (imaginary part)
q_aware_model_imag.fit(data, ffts_imag, batch_size=1024, epochs=10, validation_split=0.3)

Epoch 1/10
684/684 [==============================] - 3s 3ms/step - loss: 0.0026 - val_loss: 9.8830e-04
Epoch 2/10
684/684 [==============================] - 2s 3ms/step - loss: 7.2314e-04 - val_loss: 6.3670e-04
Epoch 3/10
684/684 [==============================] - 2s 3ms/step - loss: 5.7672e-04 - val_loss: 5.8154e-04
Epoch 4/10
684/684 [==============================] - 2s 3ms/step - loss: 5.5611e-04 - val_loss: 5.5836e-04
Epoch 5/10
684/684 [==============================] - 2s 3ms/step - loss: 5.5386e-04 - val_loss: 5.5696e-04
Epoch 6/10
684/684 [==============================] - 2s 3ms/step - loss: 5.5332e-04 - val_loss: 5.7548e-04
Epoch 7/10
684/684 [==============================] - 2s 3ms/step - loss: 5.5397e-04 - val_loss: 5.8127e-04
Epoch 8/10
684/684 [==============================] - 2s 3ms/step - loss: 5.5501e-04 - val_loss: 5.7348e-04
Epoch 9/10
684/684 [==============================] - 2s 3ms/step - loss: 5.5401e-04 - val_loss: 5.8302e-04
Epoch 10/10
684/684 [===========

# Evaluating performance

In [15]:
nbatches = 100000    # Number of batches
timesteps = 32       # Number of values in each batch
test_data = randomSignal(nbatches, timesteps_per_batch=timesteps)
test_data = np.array(data)

In [16]:
# Calculate the ground truth output of the FFT function (labels)
ffts_test = np.fft.rfft(test_data)

In [17]:
# decompose the test ffts to real parts and imaginary parts
ffts_real_test = ffts_test.real
ffts_imag_test = ffts_test.imag

In [18]:
baseline_model_real_loss = model_real.evaluate(
    test_data, ffts_real_test, verbose=1)

q_aware_model_real_loss = q_aware_model_real.evaluate(
   test_data, ffts_real_test, verbose=1)

print('Baseline test MSE:', baseline_model_real_loss)
print('Quant test MSE:', q_aware_model_real_loss)

31250/31250 [==============================] - 50s 2ms/step - loss: 9.2501e-04
Baseline test MSE: 2.7393456426827445e-12
Quant test MSE: 0.0009250055300071836


In [19]:
baseline_model_imag_loss = model_imag.evaluate(
    test_data, ffts_imag_test, verbose=1)

q_aware_model_imag_loss = q_aware_model_imag.evaluate(
   test_data, ffts_imag_test, verbose=1)

print('Baseline test MSE:', baseline_model_imag_loss)
print('Quant test MSE:', q_aware_model_imag_loss)

31250/31250 [==============================] - 48s 2ms/step - loss: 5.5886e-04
Baseline test MSE: 2.068955467018907e-12
Quant test MSE: 0.0005588564672507346


# Performing Quantization

In [20]:
# This is when the quantization takes place
converter_real = tf.lite.TFLiteConverter.from_keras_model(q_aware_model_real)
converter_real.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model_real = converter_real.convert()

INFO:tensorflow:Assets written to: /tmp/tmp0zsjawku/assets


INFO:tensorflow:Assets written to: /tmp/tmp0zsjawku/assets


In [21]:
# This is when the quantization takes place
converter_imag = tf.lite.TFLiteConverter.from_keras_model(q_aware_model_imag)
converter_imag.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model_imag = converter_imag.convert()

INFO:tensorflow:Assets written to: /tmp/tmp_ky_yigl/assets


INFO:tensorflow:Assets written to: /tmp/tmp_ky_yigl/assets
